- By: Alex Kwon
- Reference: Online Portfolio Selection: Principles and Algorithms

# Online Portfolio Selection

### Introduction

Online Portfolio Selection sequentially allocates capital among a set of assets aiming to maximize the final return of investment in a long run. OLPS plays a crucial role in a wide range of financial investment applications, such as automated wealth management, hedge fund management, and quantitative trading.

This notebook explores the OLPS module implemented in mlfinlab.

In [ ]:
import numpy as np
import pandas as pd
import os
# import from mlfinlab
import mlfinlab
from mlfinlab.online_portfolio_selection import *

%matplotlib inline
np.random.seed(42)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

### Importing Data

In [ ]:
stock_prices = pd.read_csv('../tests/test_data/stock_prices.csv', parse_dates=True, index_col='Date')
stock_prices = stock_prices.dropna(axis=1)

In [ ]:
stock_prices.head()

## Benchmarks

We will first look at the benchmark strategies.

### Buy and Hold

Invests with initial portfolio and holds until the end

We will set an arbitray initial portfolio weights to be 0.25 for the first 4 assets, and then we will allocate those inital weights according to the Buy and Hold strategy. The price data will be resampled by month.

In [ ]:
init_portfolio = np.zeros(stock_prices.shape[1])
init_portfolio[[0,1,2,3]] = 0.25
bah = BAH()
bah.allocate(stock_prices, weights=init_portfolio, resample_by='M')

all_weights returns the history of weights allocated for each time period, in this case month.

In [ ]:
bah.all_weights.head()

Notice how the weights actually deviate from the original 0.25 allocated in the beginning. This is due to the fact that the underlying asset changed its price and therefore changes our weight allocation to each asset.
Now, we will see the portfolio returns over time.

In [ ]:
bah.portfolio_return.plot()

We end up with less portfolio value than we started with because the asset prices mostly decreased.

### Best Stock

Invests all capital in the best performing stock in hindsight

In [ ]:
best_stock = BESTSTOCK()
best_stock.allocate(stock_prices, resample_by='M')

In [ ]:
print(best_stock.all_weights.iloc[[0]])
print(best_stock.all_weights.iloc[[-1]])

As seen with the previous two lines, the first and last weights are identical with the weights all 0's except for TLT. This happens because TLT is the best performing stock during this period.

In [ ]:
best_stock.portfolio_return.plot()


### Constant Rebalanced Portfolio

Rebalances to a given portfolio every period

In [ ]:
crp = CRP()
crp.allocate(stock_prices, weights=init_portfolio, resample_by='M')

If we print out all the weights below, we can notice that the weights stay constant for every time period. We are consistently rebalancing our portfolio to track the initial weights.

In [ ]:
crp.all_weights

In [ ]:
crp.portfolio_return.plot()

### Best Constant Rebalanced Portfolio

This is the best CRP strategy in hindsight. We calculate the best performing fixed portfolio weight.

In [ ]:
bcrp = BCRP()
bcrp.allocate(stock_prices, resample_by='M')

In [ ]:
bcrp.all_weights.iloc[0]

In [ ]:
bcrp.all_weights.iloc[-1]

From the above two portfolio weights for the first and last period, we see that the weights are identical with 0.89725 allocated to XLK and 0.10275 to TLT.

In [ ]:
bcrp.portfolio_return.plot()

From the plot, we can see that the BCRP produces the highest portfolio returns so far. This happens because we are calculating the weights with the given market sequence. We cannot effectively predict the future, so this will be impossible to replicate with a given past information.

## Momentum

We will now move on to momentum, which follows the winners in the past.

### Exponential Gradient

Tracks the best performing stock but also adheres to the previous portfolio value by an additional regularization term
There are three update methods that we will be using: multiplicative update, gradient projection, and expectation maximization

In [ ]:
multiplicative_update = EG(eta=0, update_rule='MU')
multiplicative_update.allocate(stock_prices, weights=init_portfolio, resample_by='M')
gradient_projection = EG(eta=.5, update_rule='GP')
gradient_projection.allocate(stock_prices, weights=init_portfolio, resample_by='M')
expectation_maximization = EG(eta=10, update_rule='EM')
expectation_maximization.allocate(stock_prices, weights=init_portfolio, resample_by='M')

If the learning rate, eta, is 0, the algorithm does not change its value, and therefore we see that the first and last weights are the same

In [ ]:
multiplicative_update.all_weights.iloc[[0,-1]]

For a higher eta, we can see that the weights diverge to track the best performing stock in the previous period.

In [ ]:
expectation_maximization.all_weights.iloc[[0,1,-2,-1]]

For a more reasonable value of eta, we see that the weights still follow the original portfolio yet follow the best stocks in the past periods.


In [ ]:
gradient_projection.all_weights.iloc[[0,1,-2,-1]]

In [ ]:
multiplicative_update.portfolio_return.plot()
gradient_projection.portfolio_return.plot()
expectation_maximization.portfolio_return.plot()

### Follow the Leader

Tracks the BCRP strategy over known periods. This is different from the original BCRP strategy in that the original strategy calcualtes the weights based on all market sequences, whereas FTL calcualtes the BCRP up to the previous trading period.

In [ ]:
ftl = FTL()
ftl.allocate(stock_prices, resample_by='M')

In [ ]:
ftl.all_weights

Just because the predicted portfolio weight was the most profitable weight in the past doesn't indicate that it will become the most profitable strategy in the future. We can see that in the below graph that simply following the best strategy does not return the best returns.

In [ ]:
ftl.portfolio_return.plot()

### Follow the Regularized Leader

FTRL mediates the FTL strategy by adding a regularization term so that the portfolio doesn't change weights too drastically for each time period

In [ ]:
ftrl = FTRL()
ftrl.allocate(stock_prices)

In [ ]:
ftrl.all_weights
